# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lefty_suit-999000
INFO:tensorflow:Restorin

### Running through an example

In [3]:

vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=2, dealer=0, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Loaded lib dds.dll


[{'call': '4H', 'insta_score': 0.404, 'expected_score': 336, 'adjustment': 20},
 {'call': 'X', 'insta_score': 0.339, 'expected_score': 290, 'adjustment': 17},
 {'call': 'PASS',
  'insta_score': 0.251,
  'expected_score': 160,
  'adjustment': 13}]

In [4]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.965}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

[]

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(['PAD_START','2N','PASS','3C','PASS','3D'], 0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
x.AKJT8x.xxx.Kxx JTxx..AK9x.AJ98x 9xxx.Q9x.QTxx.Tx AKQ8.xxxx.J8.Qxx
x.AKJT8x.xxx.Kxx AQ8x.9x.K98x.Txx T9xx..ATxx.QJ98x KJxx.Qxxxx.QJ.Ax
x.AKJT8x.xxx.Kxx K9xx.x.AJ9.AQJTx ATxx.Q9x.QTxx.8x QJ8x.xxx.K8x.9xx
x.AKJT8x.xxx.Kxx AKTxx.x.KQTx.9xx xxx.Q9xx.98xx.Tx QJ98.xx.AJ.AQJ8x
x.AKJT8x.xxx.Kxx AKQ.x.AT8x.AT98x Txxx.xx.KJx.QJxx J98xx.Q9xx.Q9x.x
x.AKJT8x.xxx.Kxx Kxxx.xx.QTx.AJxx QJ9x.Q9.KJ8x.QT8 AT8x.xxx.A9x.9xx
x.AKJT8x.xxx.Kxx AKT9x.xx.Ax.AT8x Q8x.x.QJ8xx.Q9xx Jxxx.Q9xx.KT9.Jx
x.AKJT8x.xxx.Kxx AKx.Qxx.ATx.QJ8x QJ8xx.9xx.Q8.Txx T9xx.x.KJ9xx.A9x
x.AKJT8x.xxx.Kxx AKxx.xxx.Ax.AJ9x 98xx.x.KJT9x.QTx QJTx.Q9x.Q8x.8xx
x.AKJT8x.xxx.Kxx J9xx.xx.Ax.AJT8x KTxx.9x.KQJT.Qxx AQ8x.Qxx.98xx.9x
x.AKJT8x.xxx.Kxx Kxx.Qxx.KJ8.AQ9x AJ98x.x.Qxxx.8xx QTxx.9xx.AT9.JTx
x.AKJT8x.xxx.Kxx A9x.9xx.AK.Q8xxx QJxxx.Q.QJT8.T9x KT8x.xxx.9xxx.AJ
x.AKJT8x.xxx.Kxx QJT98..KQT.AQJT9 xxxxx.Qxx.98x.8x AK.9xxx.AJxx.xxx
x.AKJT8x.xxx.Kxx KJT.Qx.A8x.JT9xx Q9xxxx.9.KQJ9.Qx A8x.xxxx.Txx.A8x
x.AKJT8x.xxx.Kxx Jxx.Q.AK98.A

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [7]:
hands = random_deal().split()
hands # N, E, S, W

['76.Q.A972.K97654',
 'A842.JT975.J64.2',
 'J53.A832.KQT8.J3',
 'KQT9.K64.53.AQT8']

In [8]:
bidder_bots = [BotBid([False, False], hand, models, sampler,i, 0,False) for i, hand in enumerate(hands)]

In [9]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS',
 'PASS',
 'PASS',
 '1C',
 'PASS',
 '1H',
 'PASS',
 '1S',
 '2C',
 '2S',
 '3C',
 'X',
 'PASS',
 'PASS',
 'PASS']